In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
censuses_data = pd.read_csv("../../CSV/censuses.csv")
censuses_data.count()

line_number               4244
house_number              4090
street_name               4090
last_name                 4090
given_name                4090
relation                  4090
color_race                4244
sex                       4244
age_at_last_birthday      4244
occupation                4244
own_or_rent               4090
owned_free_or_mortgage    4090
house_or_farm             4090
my_notes                  4090
file_name                 4090
census_year               4244
grouped_occu              4244
dtype: int64

In [3]:
# # Separate out the occupation which are null/ none (LATER we can test our model again these cases)
# occupation_missing_mask = (censuses_data["occupation"].str.contains('None')) | (censuses_data["occupation"].str.contains('Null'))
# df_missing_occupation = censuses_data[occupation_missing_mask]

# df_missing_occupation.count()

In [4]:
# prepare new df where occupation column is All labelled
# df_all_occupation = censuses_data[occupation_missing_mask == False]

# include following features that MAY influence occupation field -
# color_race, sex, age_at_last_birthday, occupation
newdata = censuses_data.loc[:,["color_race","sex","age_at_last_birthday", "grouped_occu","census_year"]]
newdata.count()

color_race              4244
sex                     4244
age_at_last_birthday    4244
grouped_occu            4244
census_year             4244
dtype: int64

In [5]:
# newdata = newdata[newdata['census_year'] != 1920]
# newdata.count()
del newdata['census_year']
newdata.count()

color_race              4244
sex                     4244
age_at_last_birthday    4244
grouped_occu            4244
dtype: int64

In [6]:
# cleanup age column
def age_clean(data,under_1_value = 1, pandas = 'yes'):
    clean_age = []
    if pandas == 'yes':
        data_lst = data.tolist()
    else:
        data_lst = data
    for datum in data_lst:
        if 'month' in str(datum):
            if 'and' in str(datum):
                clean_age.append(str(datum).split(' ')[0])
            else:
                clean_age.append(str(under_1_value))
        else:
            clean_age.append(datum)
    return clean_age

clean_age = age_clean(newdata.age_at_last_birthday,under_1_value = 1, pandas = 'yes')
newdata["age"] = clean_age

del newdata['age_at_last_birthday']
newdata.head()

,color_race,sex,grouped_occu,age
0,B,F,Unskilled,26
1,B,M,Unskilled,27
2,B,F,Unskilled,25
3,B,F,Unskilled,25
4,B,F,Student,8


In [7]:
newdata = newdata[newdata.age != 'Un.']

newdata['age'] = pd.to_numeric(newdata['age'], errors='coerce')

# put age in ranges
age_ranges= [0,11,19,40,65,110]
age_labels = ['age_0-11','age_12-19','age_20-40','age_40-65','age_Above-65']
newdata['age'] = pd.cut(newdata['age'], bins=age_ranges, labels=age_labels)
newdata['age'].unique()

[age_20-40, age_0-11, age_40-65, age_12-19, NaN, age_Above-65]
Categories (5, object): [age_0-11 < age_12-19 < age_20-40 < age_40-65 < age_Above-65]

In [8]:
newdata.head(10)

,color_race,sex,grouped_occu,age
0,B,F,Unskilled,age_20-40
1,B,M,Unskilled,age_20-40
2,B,F,Unskilled,age_20-40
3,B,F,Unskilled,age_20-40
4,B,F,Student,age_0-11
5,B,M,Unskilled,age_40-65
6,B,F,Missing Occupations,age_20-40
7,B,M,Unskilled,age_12-19
8,B,F,Missing Occupations,age_12-19
9,B,M,Student,age_12-19


In [9]:
# cleanup race column, replace M's by B & drop Jp - Just make sure we have B & W race
newdata['color_race'].replace(['Mu', 'M', 'M/W', 'My'], "B", inplace=True)
newdata = newdata[newdata.color_race != 'Jp']
newdata['color_race'].unique()

array(['B', 'W'], dtype=object)

In [10]:
# cleanup sex column - No cleanup needed , just have M & F
newdata = newdata[newdata.sex != 'FM']
newdata['sex'].unique()

array(['F', 'M'], dtype=object)

In [11]:
newdata = newdata.dropna()
newdata.count()

color_race      4231
sex             4231
grouped_occu    4231
age             4231
dtype: int64

In [12]:
# replacing the labels
# 'Unskilled'=0
# 'Skilled Labor'=1
# 'Skilled Professionals'=2
# 'Student'=3
# 'Unemployed'=4
# 'Missing Occupations'=5

newdata['grouped_occu'][newdata['grouped_occu']=='Unskilled']=0
newdata['grouped_occu'][newdata['grouped_occu']=='Skilled Labor']=1
newdata['grouped_occu'][newdata['grouped_occu']=='Skilled Professionals']=2
newdata['grouped_occu'][newdata['grouped_occu']=='Student']=3
newdata['grouped_occu'][newdata['grouped_occu']=='Unemployed']=4
newdata['grouped_occu'][newdata['grouped_occu']=='Missing Occupations']=5

newdata.head(10)

D:\Users\kshah\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\Users\kshah\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Users\kshah\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
D:\Users\kshah\

,color_race,sex,grouped_occu,age
0,B,F,0,age_20-40
1,B,M,0,age_20-40
2,B,F,0,age_20-40
3,B,F,0,age_20-40
4,B,F,3,age_0-11
5,B,M,0,age_40-65
6,B,F,5,age_20-40
7,B,M,0,age_12-19
8,B,F,5,age_12-19
9,B,M,3,age_12-19


In [13]:
newdata.to_csv("newdata_01.csv", encoding='utf-8', index=False)

In [14]:
model_data = pd.read_csv("newdata_01.csv")
model_data.count()

color_race      4231
sex             4231
grouped_occu    4231
age             4231
dtype: int64

In [15]:
# get all columns except Missing occupations & Student
# mask = ((model_data['grouped_occu'] != 5) | (model_data['grouped_occu'] != 3))
newdata = model_data[(model_data['grouped_occu'] != 5)]
newdata.grouped_occu.unique()

array([0, 3, 1, 2, 4], dtype=int64)

In [16]:
X = newdata.drop("grouped_occu", axis=1)
y = newdata["grouped_occu"]

print(X.shape, y.shape)
y.unique()

(3904, 3) (3904,)


array([0, 3, 1, 2, 4], dtype=int64)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=1/3)
print(X_train.shape, y_train.shape)

(2602, 3) (2602,)


In [18]:
X_test.count()

color_race    1302
sex           1302
age           1302
dtype: int64

In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_ohe = ohe.fit_transform(X_train)
X_test_ohe = ohe.transform(X_test)

print(X_train_ohe.shape, X_test_ohe.shape)

(2602, 9) (1302, 9)


In [20]:
y_test.unique()

array([0, 4, 1, 2, 3], dtype=int64)

In [21]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs')

classifier.fit(X_train_ohe,y_train)

# clf = LogisticRegression(random_state=0, solver='lbfgs',
#                           multi_class='multinomial').fit(X_train_ohe,y_train)

print(f"Training Data Score: {classifier.score(X_train_ohe, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_ohe, y_test)}")

Training Data Score: 0.691391237509608
Testing Data Score: 0.6612903225806451


D:\Users\kshah\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [22]:
# Save the model

import pickle
import numpy as np
import pandas as pd

In [23]:
model_data = pd.read_csv("newdata_01.csv")
model_data.count()

color_race      4231
sex             4231
grouped_occu    4231
age             4231
dtype: int64

In [24]:
filename = 'finalized_lr_model_01.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [25]:
# get all missing occupations
predictdata = model_data[model_data.grouped_occu == 5]
predictdata.head()

,color_race,sex,grouped_occu,age
6,B,F,5,age_20-40
8,B,F,5,age_12-19
10,B,M,5,age_0-11
11,B,F,5,age_0-11
13,B,M,5,age_0-11


In [26]:
X_new = pd.get_dummies(predictdata, columns =['color_race','sex', 'age'])
X_new = X_new.drop("grouped_occu", axis=1)
X_new.head()

,color_race_B,color_race_W,sex_F,sex_M,age_age_0-11,age_age_12-19,age_age_20-40,age_age_40-65,age_age_Above-65
6,1,0,1,0,0,0,1,0,0
8,1,0,1,0,0,1,0,0,0
10,1,0,0,1,1,0,0,0,0
11,1,0,1,0,1,0,0,0,0
13,1,0,0,1,1,0,0,0,0


In [27]:
# Load the model to predict the occupation
loaded_model = pickle.load(open(filename, 'rb'))

y_new = loaded_model.predict(X_new)
print(y_new)

[0 4 4 4 4 4 4 0 4 4 4 4 0 4 0 4 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 0 4 0 4 4
 0 4 4 4 0 0 0 0 0 0 0 0 4 0 0 4 4 4 4 0 0 4 4 0 4 0 4 0 4 0 4 0 4 4 0 0 0
 4 0 4 0 4 4 0 0 4 4 4 0 4 0 4 4 0 4 4 4 4 4 4 0 0 4 0 4 4 0 0 0 4 0 4 4 4
 0 0 4 4 0 0 4 4 0 4 0 0 4 4 4 0 4 0 0 4 4 4 4 4 0 0 0 4 0 0 0 4 4 0 4 4 0
 0 0 4 0 4 4 0 0 4 4 4 0 4 4 4 4 4 4 4 0 0 4 4 4 0 0 4 4 4 0 0 4 4 4 0 0 4
 0 4 0 4 4 4 4 4 4 0 0 4 0 0 0 4 4 4 4 0 0 4 0 4 4 4 4 4 4 4 0 4 4 0 4 4 4
 4 4 4 0 0 0 0 4 4 0 4 0 4 4 4 4 0 4 4 4 4 4 4 4 4 0 4 4 0 0 4 4 4 4 4 0 4
 4 4 4 4 0 4 4 4 4 4 4 4 4 4 0 4 4 4 0 4 4 0 4 4 4 4 0 4 4 4 4 0 0 4 4 4 4
 0 4 4 4 4 0 0 0 0 4 4 4 0 0 4 4 4 0 4 4 4 4 4 4 0 0 4 0 0 0 4]


In [28]:
type(y_new)

numpy.ndarray

In [29]:
X_new['predicted_occupation'] = y_new
X_new

,color_race_B,color_race_W,sex_F,sex_M,age_age_0-11,age_age_12-19,age_age_20-40,age_age_40-65,age_age_Above-65,predicted_occupation
6,1,0,1,0,0,0,1,0,0,0
8,1,0,1,0,0,1,0,0,0,4
10,1,0,0,1,1,0,0,0,0,4
11,1,0,1,0,1,0,0,0,0,4
13,1,0,0,1,1,0,0,0,0,4
14,1,0,1,0,1,0,0,0,0,4
15,1,0,1,0,1,0,0,0,0,4
22,1,0,1,0,0,0,0,1,0,0
26,1,0,1,0,0,1,0,0,0,4
27,1,0,0,1,1,0,0,0,0,4
